In [75]:
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('monokai')

There were lessons learnt from Amazon, Sourashtramatrimoni, Linkedin websites. There were unique challenges in each cases and the scraping was not exactly very successful. 

There are couple of ideas still not fully explored, 

    Using XSLT or XML libraries to scrape the sources

    Using Xpath to pinpoint the text required

The challenge is only in locating a reliable reference point to the texts in the website sources. It is about time I used the X_path and X_query effectively for my next project. 

The above ideas has been made real in this notebook. The Xpath usage has been greatly streamlined and understood. Following that, much of the ReactJS based website has been scraped with help of Xpath methods

Then extend that to earlier implementations...

Further working on multiple websites, there are couple of insights and processes that has emerged

- Start with the Selenium driver, since it returns the webpage source more reliably

- Check for the last tag that is near your target text or link in the site. 

- Reach the text or link with the minimum levels of xpath by using the attributes and contains method

- Iterate over the list of data scraped and loop over the same

In [114]:
#Initiating the Selenium Drivers
from selenium import webdriver
import time as time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.chrome.options import Options
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dataset
from bs4 import BeautifulSoup 
from lxml import html
import requests
from urllib.parse import unquote
from urllib.parse import urljoin

In [4]:
entry = 'https://www.messari.io'

In [6]:
driver = webdriver.Chrome()
driver.get(entry)
driver.maximize_window()

In [14]:
html_source = driver.page_source
soup_net = BeautifulSoup(html_source,'html.parser')
driver.quit()

In [11]:
soup_lxml = BeautifulSoup(html_source,'lxml')

In [15]:
tree = html.fromstring(html_source) #use the source directly

In [16]:
#This is the long process
#tree.xpath('//section[2]/div/div[2]/div[2]/table/tbody/tr/td/div/span/a/text()')
#crypto_links = tree.xpath('//section[2]/div/div[2]/div[2]/table/tbody/tr/td/div/span/a/@href')

#Trying the shorter Xpath robust process
tbod = tree.xpath('//tbody[contains(@class,"MuiTableBody-root")]')[0]

ecosystem = []
for crypto in tbod.xpath('//a/@href')[11:42]:
    ecosystem.append(urljoin('https://messari.io/',crypto))

In [17]:
cryptos = tbod.xpath('//a/text()')[6:36]
len(cryptos)

ecosystem.pop(15) # It ejects one of the elements from the list
len(ecosystem)

30

In [19]:
#Creating the dataframes with the links and the cryptos/ ecosystem to explore
messari_crawler = pd.DataFrame({'ecosystems':cryptos,'links':ecosystem})
messari_crawler.head()

,ecosystems,links
0,Bitcoin,https://messari.io/asset/bitcoin
1,Ethereum,https://messari.io/asset/ethereum
2,Tether,https://messari.io/asset/tether
3,BNB,https://messari.io/asset/binance-coin
4,USD Coin,https://messari.io/asset/usd-coin


In [21]:
#none of the below work to get the data required, since the website is ReactJS basis
tree.xpath('//table/@class')
tree.xpath('//section/@class')[:5]
tree.xpath('//span[.="Top Assets"]/text()')
tree.xpath('//div/@class=jss38') 
tree.xpath('//div/@class')[:10]
tree.xpath('//table/tbody[@class="MuiTable-root jss192 jss750"]')
#expression is correct, but no resullt...
tree.xpath('//thead/@*')

['MuiTableHead-root jss148 jss205 css-1wbz3t9 e1de0imv0',
 'MuiTableHead-root jss148 jss192 css-1wbz3t9 e1de0imv0',
 'MuiTableHead-root jss148 jss271 css-1wbz3t9 e1de0imv0',
 'MuiTableHead-root jss148 jss154 jss149 css-1wbz3t9 e1de0imv0',
 'MuiTableHead-root jss148 jss242 jss149 css-1wbz3t9 e1de0imv0',
 'MuiTableHead-root jss148 jss183 css-1wbz3t9 e1de0imv0',
 'MuiTableHead-root jss148 jss218 css-1wbz3t9 e1de0imv0',
 'MuiTableHead-root jss148 jss249 css-1wbz3t9 e1de0imv0',
 'MuiTableHead-root jss148 jss260 css-1wbz3t9 e1de0imv0',
 'MuiTableHead-root jss148 jss229 css-1wbz3t9 e1de0imv0']

In [54]:
def selenium_loader(link):
    driver = webdriver.Chrome()
    driver.get(link)
    driver.maximize_window()
    driver.refresh()
    time.sleep(5)
    html_source = driver.page_source
    tree = html.fromstring(html_source)
    driver.quit()
    return tree

In [115]:
def selenium_headless(link):
    options = Options()
    options.headless = True #making this driver headless so it can work in the background
    driver = webdriver.Chrome(options=options, executable_path=r'chromedriver.exe')
    driver.get(link)
    driver.maximize_window()
    driver.refresh()
    time.sleep(5)
    html_source = driver.page_source
    tree = html.fromstring(html_source)
    driver.quit()
    return tree

1) Find the various major tables or sections on the profile page
    Extract the information from that page section /table

2) Find the tabs on the profile page. 
    get their links

3) Find the Key Areas and list them
    Get the details under key metrics, key info and summary



![](2022-02-21-16-28-05.png)

In [23]:
main_page = selenium_headless(messari_crawler.links[0])

In [24]:
#failed attempts at getting the top_tabs, don't run

main_page.xpath('//span/text()')

tabs = main_page.xpath('//span/text()')
for tab in tabs[12:21]:
    print(tab)

Real Volume (24H)
$4.84B
Marketcap
$794B
Y+10 Marketcap
$865B
Y2050 Marketcap
$878B
ATH


Tabs to be clicked on every page by the driver, when it crawls are

Overview, 
Profile
Markets
Metrics
Research
News

In [108]:
def get_over_view(page_source):
    tabs = main_page.xpath('//a[@role="tab"]')
    over_view = []
    for tab in tabs[1:9]:
        over_view.append(tab.get('href'))    
    over_view = over_view[:3]+over_view[5:-1]
    return over_view

In [109]:
crypto_page = get_over_view(main_page)

In [ ]:
crypto_tree = []
for crypto in crypto_page:
    #join the individual links and get their tree
    temp = urljoin(entry,crypto)
    print('Getting *** {} *** tree for scraping'.format(temp))
    crypto_tree.append(selenium_headless(temp))

Getting *** https://www.messari.io/asset/bitcoin/profile *** tree for scraping
Getting *** https://www.messari.io/asset/bitcoin/markets *** tree for scraping
Getting *** https://www.messari.io/asset/bitcoin/metrics *** tree for scraping
Getting *** https://www.messari.io/asset/bitcoin/research *** tree for scraping


['/asset/bitcoin/profile/roadmap',
 '/asset/bitcoin/profile/contributors',
 '/asset/bitcoin/profile/investors',
 '/asset/bitcoin/profile/token-usage',
 '/asset/bitcoin/profile/technology',
 '/asset/bitcoin/profile/governance',
 '/asset/bitcoin/profile/regulation']

In [116]:
#The tree of each web page has been extracted. Now the web-page needs to be scraped for the data.
#TOC is the source of all the information in the webpage, extracting the links
def get_ov_and_toc(tree_0,entry):
    toc_1 = crypto_tree[0].xpath('//a[@class]/@href')[27:34]
    over_text = crypto_tree[0].xpath('//p/text()')
    toc_tree = []
    for toc in toc_1:
        full = urljoin(entry,toc)
        print('Extract {} data'.format(full))
        toc_tree.append(selenium_headless(full))
    return over_text, toc_tree
        

In [29]:
#The ends-with option is not working Chrome-browser so need to use substring, not working
span_data = main_page.xpath('//span["ion"=substring(@class,string-length(@class),string-length(@class)-3)]')
#//div[substring(@title, string-length(@title) - 2) = 'gle']
#//input['name'=substring(@name,5,8)]
for dat in span_data:
    print(dat.text)

In [32]:
def get_overview(over_page):
#using the simple array manipulation to get the data
    profile_columns = [] #The columns on which the values will be filled 
    span_data = list(main_page.xpath('//div[contains(@class,"undefined")]/div/div/span[contains(@class,"jss")]/text()'))
    span_values = list(main_page.xpath('//div[contains(@class,"undefined")]/div/div/span[not(contains(@class,"jss"))]/text()'))
    span_data = span_data[:10]+span_data[12:13]+span_data[15:-1]
    span_values = span_values[:-4]
    if len(span_data) == len(span_values):
        print('data and its values match, returning as df')
        temp_df = pd.DataFrame(zip(span_data,span_values)).T #Used the zip method and Transpose option
        return temp_df
    else:
        print('Issue with scraped values, check before use')
        return span_data, span_values

In [33]:
data = get_overview(main_page)

data and its values match, returning as df


In [103]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,Price,1H Range,24H Range,Real Volume (24H),Marketcap,Y+10 Marketcap,Y2050 Marketcap,ATH,ATH Date,Down from ATH,Cycle Low Date,Sector,Token Type,Token Usage,Consensus Algorithm,Precise Consensus Algorithm,Genesis Block Date
1,"$41,943.91","$41,779.88 - $41,790.33","$40,801.02 - $40,964.50",$4.84B,$794B,$865B,$878B,"$68,721.93","November 10th, 2021","$33,116.00","January 24th, 2022",Currencies,Native,Payments,Proof-of-Work,Nakamoto Consensus,"January 3rd, 2009"


In [112]:
get_ov_and_toc(tree[0],entry)

Extract https://www.messari.io/asset/bitcoin/profile/roadmap data
Extract https://www.messari.io/asset/bitcoin/profile/contributors data
Extract https://www.messari.io/asset/bitcoin/profile/investors data
Extract https://www.messari.io/asset/bitcoin/profile/token-usage data
Extract https://www.messari.io/asset/bitcoin/profile/technology data
Extract https://www.messari.io/asset/bitcoin/profile/governance data
Extract https://www.messari.io/asset/bitcoin/profile/regulation data


(['A peer-to-peer electronic cash system',
  'Bitcoin is the first ',
  ", censorship-resistant, permissionless, peer-to-peer payment settlement network with a provably scarce, programmable, native currency. Bitcoin (BTC), the native asset of the Bitcoin blockchain, is the world's first digital currency without a central bank or administrator. The Bitcoin network is an emergent decentralized monetary institution that exists through the interplay between ",
  ', ',
  ', and developers. It is set by a social contract that is created and opted into by the users of the network and hardened through ',
  ' and ',
  '. Bitcoin is the first, oldest, and largest cryptocurrency in the world.',
  '\nBitcoin borrows from nearly three decades of research by ',
  ', ',
  ', and hobbyists who tried to create similar payment protocols. Some of the most prominent projects that preceded Bitcoin include: Digicash, b-money, Hashcash, e-gold, and Bit Gold. Bitcoin is often associated with Austrian economic

In [95]:
profile_columns

['Governor',
 'new',
 '$42,044.31',
 '(+2.68%)',
 None,
 None,
 '$42,044.31',
 '(+2.68%)',
 None,
 None,
 'Price',
 '1H Range',
 '24H Range',
 'Real Volume (24H)',
 'Marketcap',
 'Y+10 Marketcap',
 'Y2050 Marketcap',
 'ATH',
 'ATH Date',
 'Down from ATH',
 '-38.82%',
 'Cycle Low',
 'Cycle Low Date',
 'Up From Cycle Low',
 '+26.96%',
 None,
 None,
 'Period',
 None,
 'VS USD',
 None,
 'VS ETH',
 None,
 'VS BTC',
 None,
 'VS DeFi',
 '1H',
 '+0.53%',
 '+0.04%',
 '0.00%',
 '+0.33%',
 '24H',
 '+2.68%',
 '-0.79%',
 '-0.78%',
 '-2.25%',
 '7D',
 '+7.90%',
 '-6.59%',
 '0.00%',
 '-6.32%',
 '1M',
 '+3.28%',
 '+0.72%',
 '0.00%',
 '-2.34%',
 '3M',
 '-10.33%',
 '+18.62%',
 '0.00%',
 '+16.95%',
 '1Y',
 '-27.86%',
 '-56.01%',
 '0.00%',
 '+6.91%',
 None,
 'Sector',
 'Token Type',
 'Token Usage',
 'Consensus Algorithm',
 'Precise Consensus Algorithm',
 'Genesis Block Date',
 None,
 None,
 '+4.34%',
 None,
 None,
 'Date',
 None,
 'High',
 None,
 'Low',
 None,
 'Volume (24H)',
 'Mar 19, 2022',
 '$42,103.98